# QCoDeS Example with Agilent 34400A

In [ ]:
%matplotlib nbagg
import matplotlib.pyplot as plt
import time
import numpy as np

import qcodes as qc
from qcodes.utils.validators import Enum, Strings
import qcodes.instrument_drivers.tektronix.Keithley_2600_channels as keith
import qcodes.instrument_drivers.agilent.Agilent_34400A as agi

from qcodes import Measurement
#from qcodes.loops import Loop

In [2]:
import time
class Timer(object):
    def __init__(self, name=None):
        self.name = name

    def __enter__(self):
        self.tstart = time.time()

    def __exit__(self, type, value, traceback):
        if self.name:
            print('[%s]' % self.name,)
        print('Elapsed: %s' % (time.time() - self.tstart))

In [ ]:
# create Instruments
import qcodes.instrument.sims as sims
path_to_yaml = sims.__file__.replace('__init__.py', 'Keithley_2600.yaml')
k1 = keith.Keithley_2600('Keithley1', address='GPIB::1::INSTR', visalib=path_to_yaml + '@sim')
#k1 = keith.Keithley_2600('Keithley1', 'GPIB0::15::INSTR')

a1 = agi.Agilent_34400A('Agilent1', 'GPIB0::11::INSTR', setter_instr=k1)
a2 = agi.Agilent_34400A('Agilent2', 'GPIB0::6::INSTR', setter_instr=k1)

# set integration time (number of line cycles)
a1.NPLC.set(10)
a2.NPLC.set(10)

In [ ]:
station1 = qc.Station()
station1.add_component(k1)
station1.add_component(a1)

meas1 = Measurement(station=station1)
meas1.register_parameter(k1.a.volt)
meas1.register_parameter(a1.volt)

meas1.write_period = 2

In [ ]:
station2 = qc.Station()
station2.add_component(k1)
station2.add_component(a1)
station2.add_component(a2)

meas2 = Measurement(station=station2)
meas2.register_parameter(k1.a.volt)
meas2.register_parameter(a1.volt)
meas2.register_parameter(a2.volt)

meas2.write_period = 2

In [ ]:
# Time single readings
with Timer('Time a1'):
    a1.volt.get()
with Timer('Time a2'):
    a2.volt.get()

In [ ]:
with Timer('Meas Time 1'):
    with meas1.run() as datasaver:
        for set_v in np.linspace(0, 25, 10):
            k1.a.volt.set(set_v)
            get_v = a1.volt.get()
            datasaver.add_result((k1.a.volt, set_v), (a1.volt, get_v))

        dataset1 = datasaver.dataset
    #data = Loop(k1.a.volt[-5:5:1], 0).each(a1.volt).run(location='testsweep', overwrite=True,background=False)

with Timer('Meas Time 2'):
    with meas2.run() as datasaver:
        for set_v in np.linspace(0, 25, 10):
            k1.a.volt.set(set_v)
            get_v1 = a1.volt.get()
            get_v2 = a2.volt.get()
            datasaver.add_result((k1.a.volt, set_v), (a1.volt, get_v1), (a2.volt, get_v2))

        dataset2 = datasaver.dataset
    #data = Loop(k1.a.volt[-5:5:1], 0).each(a1.volt, a2.volt).run(location='testsweep', overwrite=True,background=False)